[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/a-parida12/RNN_Trials/blob/main/IntroRNNs-many2One.ipynb)

**RNN based MNIST CLASSIFIER**
- Many to Many RNN

<img src="https://github.com/a-parida12/RNN_Trials/blob/main/images/rnn_orig_2.png?raw=true" alt="RNN Architecture"/>



In [1]:
import torch
import torchvision 
import torch.optim as optim 
import torch.nn as nn  
import torch.nn.functional as F  
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets  
import torchvision.transforms as transforms  
from tqdm.notebook import trange, tqdm
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Hyperparameters for MNIST
# MNIST data is 1x28x28

# sequence params
input_size = 28
sequence_length = 28
hidden_size = 256 
num_layers = 1 # no of stacked RNNs

# output params
num_classes = 10

# train params
learning_rate = 0.005
batch_size = 64
num_epochs = 1

In [3]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_directions = 1 # num_directions = 2 for bidirectional RNNs and 1 otherwise.
        #input= N x sequence_length x Features
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        #(num_layers * num_directions, batch, hidden_size)
        h0 = torch.zeros(self.num_layers*self.num_directions, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, hidden = self.rnn(x, h0)
        #import pdb; pdb.set_trace()

        #out shape->(batch, seq_len, num_directions * hidden_size) # output of the RNN from all timesteps
        #hidden -> (num_layers * num_directions, batch, hidden_size) # hidden value from the last time-step

        # Decode the hidden state of the last time step
        out = self.fc(out[:,-1,:])
        return out




In [4]:
# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)

test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
# Train Network
for epoch in trange(num_epochs, desc="Epochs"):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader, desc="Iter")):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

In [6]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with \
              accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )
    # Set model back to train
    model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 42650 / 60000 with               accuracy 71.08
Checking accuracy on test data
Got 7224 / 10000 with               accuracy 72.24
